### Quiero - 

1. agarrar todas las imagenes
2. modificar el prompt y la respuesta para que tambien me diga
 - punto en la imagen para tomar de referencia como nariz, frente, etc.
3. Bajar la resolucion de imagenes en carpeta
4. ordenar
5. recortar para encontrar el centro o punto para que se vean bien

https://platform.openai.com/docs/guides/images-vision?api-mode=responses

# 0 setup

In [13]:
import os
import time
from dotenv import load_dotenv
from openai import OpenAI
import base64
from PIL import Image, ImageOps
import glob
import os
import json

import pandas as pd

In [14]:
load_dotenv()

# Inicializa el cliente

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)
OPENAI_ASSISTANT_ID = os.getenv("OPENAI_ASSISTANT_ID") 

In [ ]:
SRC_DIR = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/sineditar"
DST_DIR = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/resized"
os.makedirs(DST_DIR, exist_ok=True)

In [16]:

TARGET_WIDTH = 1350

def resize_image(src_path, dst_path, target_width):
    img = Image.open(src_path)
    img = ImageOps.exif_transpose(img)
    w_percent = (target_width / img.width)
    target_height = int(img.height * w_percent)
    img_resized = img.resize((target_width, target_height), Image.LANCZOS)
    img_resized.save(dst_path)
    img.close()

# Solo extensiones válidas
valid_extensions = ('.jpg', '.jpeg')

for img_path in glob.glob(os.path.join(SRC_DIR, "*")):
    if img_path.lower().endswith(valid_extensions):
        filename = os.path.basename(img_path)
        dst_path = os.path.join(DST_DIR, filename)
        resize_image(img_path, dst_path, TARGET_WIDTH)

In [20]:
# Función para codificar una imagen a base64
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# 1 procesar img

In [24]:
prompt = """
Recibe un lote de imágenes del mismo sujeto y realiza dos tareas por imagen:

1 - Clasificación facial:
Posición de la cabeza: Elige una sola opción entre las siguientes:
- Perfil izquierda
- Perfil derecha
- Girado izquierda
- Girado derecha
- Frente
- NA (invalido si no se puede determinar o no es ninguna de las anteriores)

Expresión facial: Elige una sola opción entre las siguientes:
- Sin dentadura
- Con dentadura
- NA (invalido si no se puede determinar o no es ninguna de las anteriores)

2 - Detección de puntos clave:
Identifica las coordenadas (x, y) en píxeles de al menos los siguientes puntos anatómicos visibles:
- Punta de la nariz
- Centro de la boca (entre los labios)
- Centro de la frente (aproximadamente entre las cejas)
- Calificación de la imagen: Un número entre 0 y 1 que indica la calidad de la imagen, donde 0 es mala calidad y 1 es excelente calidad.

Formato de salida:

Por cada imagen, genera un objeto JSON con las siguientes claves:
- nombre_archivo: string o "NA"
- posición: string o "NA"
- expresión: string o "NA"
- nariz: [x, y] o "NA"
- boca: [x, y] o "NA"
- frente: [x, y] o "NA"
- calificación: número entre 0 y 1 o "NA"

Responde únicamente con un array JSON con todos los objetos, sin texto adicional ni explicaciones.
Se te proporciona primero el nombre de la imagen y luego la imagen en sí. El nombre de la imagen es el primer elemento del array, seguido de la imagen.
La imagen está en formato base64 y no contiene encabezados ni etiquetas adicionales.
"""

In [26]:

# Construir contenido del input
content = [{"type": "input_text", "text": prompt}]

# Leer imágenes de la carpeta y codificarlas
for filename in os.listdir(DST_DIR):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(DST_DIR, filename)
        image_base64 = encode_image_to_base64(image_path)
        # Incluir el nombre de la imagen en el contenido del mensaje de texto
        content.append({
            "type": "input_text",
            "text": f"Imagen: {filename}"
        })
        content.append({
            "type": "input_image",
            "image_url": f"data:image/jpeg;base64,{image_base64},",
            "detail": "high"
        })

# Construir input final
input_data = [
    {
        "role": "user",
        "content": content
    }
]


In [27]:
# Crear cliente
client = OpenAI()

# Hacer la solicitud
response = client.responses.create(
    model="o4-mini",
    reasoning={"effort": "high"},
    input=[
        {
            "role": "user",
            "content": content
        }
    ],
)

KeyboardInterrupt: 

In [ ]:
print(response.output[1].content[0].text)

[  
  {  
    "nombre_archivo": "IMG_4826.JPG",  
    "posición": "Perfil izquierda",  
    "expresión": "Sin dentadura",  
    "nariz": [150, 550],  
    "boca": [150, 650],  
    "frente": [150, 450],  
    "calificación": 0.9  
  },  
  {  
    "nombre_archivo": "IMG_4833.JPG",  
    "posición": "Girado derecha",  
    "expresión": "Sin dentadura",  
    "nariz": [800, 300],  
    "boca": [800, 420],  
    "frente": [800, 200],  
    "calificación": 0.9  
  },  
  {  
    "nombre_archivo": "IMG_4827.JPG",  
    "posición": "Girado izquierda",  
    "expresión": "Sin dentadura",  
    "nariz": [350, 300],  
    "boca": [350, 420],  
    "frente": [350, 200],  
    "calificación": 0.9  
  },  
  {  
    "nombre_archivo": "IMG_4831.JPG",  
    "posición": "Girado derecha",  
    "expresión": "Sin dentadura",  
    "nariz": [700, 360],  
    "boca": [700, 450],  
    "frente": [700, 230],  
    "calificación": 0.9  
  },  
  {  
    "nombre_archivo": "IMG_4825.JPG",  
    "posición": "G

In [ ]:
# Obtener el texto JSON de la respuesta
json_text = response.output[1].content[0].text

# Cargar el texto como lista de objetos JSON
data = json.loads(json_text)

# Contar cuántos objetos hay en la lista
print("Cantidad de elementos:", len(data))

Cantidad de elementos: 12


# 2 procesar img

In [21]:
prompt = """
Recibe una sola imagen de un sujeto y realiza dos tareas:

0. Ojos visibles:
   - Cuenta cuántos ojos se aprecian:
     * Si hay 2 ojos visibles ➔ sigue al paso 1
     * Si hay 1 ojo visible ➔ posición = “Perfil izquierda” si mira a la izquierda desde la perspectiva de nosotros o 
        “Perfil derecha” si mira a la derecha desde la perspectiva de nosotros 

1. Clasificación facial (solo si hay 2 ojos o no aplicó paso 0):
   - Posición de la cabeza (elige UNA):
     * Frente (rostro frontal, ambos ojos simétricos)
     * Girado izquierda (ambos ojos visibles, cabeza girada hacia la izquierda, >50% del rostro)
     * Girado derecha (ambos ojos visibles, cabeza girada hacia la derecha, >50% del rostro)
     * NA (no se puede determinar)

   - Expresión facial (elige UNA):
     * Sin dentadura (boca cerrada o apenas entreabierta sin dientes visibles)
     * Con dentadura (dentadura visible claramente)
     * NA (no se puede determinar)

2. Detección de puntos clave:
   - nariz: [x, y] de la punta de la nariz, o "NA" si no es detectable
   - calificación: número real entre 0 y 1 que evalúa la calidad de la imagen

Formato de salida ÚNICAMENTE un array JSON con un objeto así:

```json
[
  {
    "nombre_archivo": "IMG_1234.JPG",
    "posicion": "Perfil izquierda",
    "expresion": "Sin dentadura",
    "nariz": [150, 550],
    "calificacion": 0.85
  }
]
"""

In [22]:
# Lista para guardar los resultados
results = []

# Iterar sobre las imágenes
for filename in os.listdir(DST_DIR):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(DST_DIR, filename)
        image_base64 = encode_image_to_base64(image_path)

        # Construir input para esta imagen
        input_data = [
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": f"{prompt}\nImagen: {filename}"},
                    {
                        "type": "input_image",
                        "image_url": f"data:image/jpeg;base64,{image_base64}"
                    }
                ]
            }
        ]

        # Hacer la solicitud
        response = client.responses.create(
            model="o4-mini",
            reasoning={"effort": "high"},
            input=input_data
        )

        # Procesar respuesta
        if hasattr(response, 'output'):
            result = response.output[1].content[0].text  # Ajusta si cambia la estructura
            try:
                # Intenta convertir a diccionario
                data = eval(result) if isinstance(result, str) else result
                if isinstance(data, dict):
                    results.append(data)
                elif isinstance(data, list):
                    results.extend(data)
            except Exception as e:
                print(f"Error procesando la respuesta de {filename}: {e}")

# Crear DataFrame
df = pd.DataFrame(results)

# Mostrar un preview
print(df.head())


  nombre_archivo          posicion      expresion       nariz  calificacion
0   IMG_4826.JPG  Perfil izquierda             NA  [400, 500]          0.92
1   IMG_4833.JPG    Girado derecha  Sin dentadura  [590, 750]          0.92
2   IMG_4827.JPG  Perfil izquierda  Sin dentadura  [300, 700]          0.95
3   IMG_4831.JPG  Perfil izquierda             NA  [950, 550]          0.90
4   IMG_4825.JPG            Frente  Con dentadura  [384, 480]          0.95


In [23]:
df

,nombre_archivo,posicion,expresion,nariz,calificacion
0,IMG_4826.JPG,Perfil izquierda,NA,"[400, 500]",0.92
1,IMG_4833.JPG,Girado derecha,Sin dentadura,"[590, 750]",0.92
2,IMG_4827.JPG,Perfil izquierda,Sin dentadura,"[300, 700]",0.95
3,IMG_4831.JPG,Perfil izquierda,NA,"[950, 550]",0.90
4,IMG_4825.JPG,Frente,Con dentadura,"[384, 480]",0.95
5,IMG_4824.JPG,Frente,Sin dentadura,"[720, 900]",0.95
6,IMG_4830.JPG,Perfil izquierda,NA,"[600, 800]",0.95
7,IMG_4834.JPG,Girado derecha,Con dentadura,"[1450, 370]",0.92
8,IMG_4835.JPG,Perfil derecha,NA,"[820, 500]",0.92
9,IMG_4823.JPG,Frente,Sin dentadura,"[384, 480]",0.98


In [24]:
df.to_csv('/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/notebooks/data_01.csv', index=False)

# 3 ordenar imagenes

In [5]:
from reportlab.lib.pagesizes import A4, landscape
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm
from PIL import Image
import pandas as pd
import os

# --- Rutas de archivos ---
csv_path = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/notebooks/data.csv"
img_folder = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/sineditar"
output_pdf = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/output_fotos_actualizado.pdf"

# --- Leer CSV ---
df = pd.read_csv(csv_path)

# === PÁGINA 1 (vertical) ===
c = canvas.Canvas(output_pdf, pagesize=A4)
w, h = A4

# 1. Buscar la foto frontal con dentadura
frontal = df[(df['posicion'] == 'Frente') & (df['expresion'] == 'Con dentadura')].iloc[0]
img_path = os.path.join(img_folder, frontal['nombre_archivo'])

# 2. Rotar si la imagen está en horizontal
img = Image.open(img_path)
if img.width > img.height:
    img = img.rotate(90, expand=True)
    temp_path = os.path.join(img_folder, f"rotated_{frontal['nombre_archivo']}")
    img.save(temp_path)
    img_path = temp_path

# 3. Dibujar la imagen centrada (tamaño fijo de 12 cm de ancho)
img_width = 12 * cm
aspect = img.height / img.width
img_height = img_width * aspect
c.drawImage(img_path, (w - img_width) / 2, h - img_height - 6*cm, width=img_width, height=img_height)

# 4. Texto debajo de la imagen
c.setFont("Helvetica", 16)
c.drawCentredString(w / 2, h - img_height - 7.5*cm, "SEBASTIAN MORA AGUILERA")
c.setFont("Helvetica", 12)
c.drawString(2*cm, h - img_height - 8.7*cm, "EDAD: 10 AÑOS")
c.drawString(8*cm, h - img_height - 8.7*cm, "FECHA DE NACIMIENTO: 08/03/2015")
c.drawString(2*cm, h - img_height - 9.5*cm, "FECHA DE TOMA: 17/05/2025")
c.drawString(2*cm, h - img_height - 10.3*cm, "DRA. CLAUDIA RIVERO MARIN")

c.showPage()

# === PÁGINA 2 (horizontal) ===
c.setPageSize(landscape(A4))
w, h = landscape(A4)

# 1. Definir orden y separar filas
orden = ["Perfil izquierda", "Girado izquierda", "Frente", "Girado derecha", "Perfil derecha"]
sin_dent = df[df['expresion'] == "Sin dentadura"].copy()
con_dent = df[df['expresion'] == "Con dentadura"].copy()

def get_ordered(filtrado):
    resultados = []
    for pos in orden:
        match = filtrado[filtrado['posicion'] == pos]
        if not match.empty:
            resultados.append(match.iloc[0]['nombre_archivo'])
    return resultados

row1 = get_ordered(sin_dent)  # Sin dentadura
row2 = get_ordered(con_dent)  # Con dentadura

# 2. Cálculo de coordenadas para 5 imágenes en cada fila
x_start = 1 * cm
img_size = 6 * cm
gap = (w - 2*cm - (5 * img_size)) / 3.5  # más espacio horizontal


# 3. Dibujar la primera fila (sin dentadura)
y1 = h - 4*cm - img_size
for i, filename in enumerate(row1):
    img_path = os.path.join(img_folder, filename)
    if os.path.exists(img_path):
        img = Image.open(img_path)
        if img.width > img.height:
            img = img.rotate(90, expand=True)
            temp_path = os.path.join(img_folder, f"rotated_{filename}")
            img.save(temp_path)
            img_path = temp_path
        x = x_start + i * (img_size + gap)
        c.drawImage(img_path, x, y1, width=img_size, height=img_size)

# 4. Dibujar la segunda fila (con dentadura)
y2 = y1 - img_size - 1.8*cm 
for i, filename in enumerate(row2):
    img_path = os.path.join(img_folder, filename)
    if os.path.exists(img_path):
        img = Image.open(img_path)
        if img.width > img.height:
            img = img.rotate(90, expand=True)
            temp_path = os.path.join(img_folder, f"rotated_{filename}")
            img.save(temp_path)
            img_path = temp_path
        x = x_start + i * (img_size + gap)
        c.drawImage(img_path, x, y2, width=img_size, height=img_size)

c.save()


# 2.5 CROP

In [33]:
from PIL import Image, ImageOps
import ast

def resize_image(src_path, target_width):
    img = Image.open(src_path)
    img = ImageOps.exif_transpose(img)
    w_percent = (target_width / img.width)
    target_height = int(img.height * w_percent)
    img_resized = img.resize((target_width, target_height), Image.LANCZOS)
    return img_resized

def crop_around_nose(img, nose_coord, crop_width=600, crop_height=800):
    x, y = nose_coord
    left = max(x - crop_width // 2, 0)
    upper = max(y - crop_height // 2, 0)
    right = min(left + crop_width, img.width)
    lower = min(upper + crop_height, img.height)
    return img.crop((left, upper, right, lower))


In [34]:
# 1. Obtener nombre del archivo y nariz desde el CSV
row = df[df['nombre_archivo'] == 'IMG_4827.JPG'].iloc[0]
img_path = os.path.join(img_folder, row['nombre_archivo'])

# 2. Convertir coordenada a tupla
nose = ast.literal_eval(row['nariz'])

# 3. Redimensionar
img_resized = resize_image(img_path, 1350)

# 4. Recortar alrededor de la nariz
img_cropped = crop_around_nose(img_resized, nose, 800, 1300)

# 5. Guardar temporal para usar en el PDF
img_cropped_path = os.path.join(img_folder, "cropped_IMG_4827.JPG")
img_cropped.save(img_cropped_path)


In [50]:
import os
import ast
import pandas as pd
from PIL import Image, ImageOps

# --- Configuración ---
csv_path = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/notebooks/data.csv"
img_folder = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/sineditar"
output_folder = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/fotos_finales"
resized_width_csv = 1350

# Diccionario: márgenes personalizados por posición
recortes_por_posicion = {
    "Perfil izquierda":    {"left": 200, "right": 1000, "top": 1400, "bottom": 950},
    "Girado izquierda":    {"left": 300, "right": 1000, "top": 1400, "bottom": 950},
    "Frente":              {"left": 1000, "right": 1000, "top": 1400, "bottom": 950},
    "Girado derecha":      {"left": 1000, "right": 300, "top": 1400, "bottom": 950},
    "Perfil derecha":      {"left": 1000, "right": 200, "top": 1400, "bottom": 950}
}

posiciones_deseadas = list(recortes_por_posicion.keys())
expresiones_deseadas = ["Sin dentadura", "Con dentadura"]

# Crear carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Leer CSV
df = pd.read_csv(csv_path)

# --- Función de recorte dinámico ---
def crop_relative_to_nose(img, nose_coord, l, r, t, b):
    x, y = nose_coord
    left = max(x - l, 0)
    upper = max(y - t, 0)
    right = min(x + r, img.width)
    lower = min(y + b, img.height)
    return img.crop((left, upper, right, lower))

# --- Procesamiento principal ---
for expresion in expresiones_deseadas:
    for pos in posiciones_deseadas:
        subset = df[(df['expresion'] == expresion) & (df['posicion'] == pos)]
        if not subset.empty:
            row = subset.iloc[0]
            nombre = row['nombre_archivo']
            nariz_resized = ast.literal_eval(row['nariz'])
            src_path = os.path.join(img_folder, nombre)

            if os.path.exists(src_path):
                try:
                    img = Image.open(src_path)
                    img = ImageOps.exif_transpose(img)

                    # Escalar coordenadas
                    scale_factor = img.width / resized_width_csv
                    nariz_original = (
                        int(nariz_resized[0] * scale_factor),
                        int(nariz_resized[1] * scale_factor)
                    )

                    # Obtener márgenes personalizados
                    recorte = recortes_por_posicion[pos]
                    img_cropped = crop_relative_to_nose(
                        img,
                        nariz_original,
                        recorte["left"],
                        recorte["right"],
                        recorte["top"],
                        recorte["bottom"]
                    )

                    # Guardar
                    nombre_base = f"cropped_{expresion.replace(' ', '_')}_{nombre}"
                    out_path = os.path.join(output_folder, nombre_base)
                    img_cropped.save(out_path)
                    print(f"Recortada: {nombre} | {pos} | Expresión: {expresion}")
                except Exception as e:
                    print(f"Error con {nombre}: {e}")
            else:
                print(f"No encontrada: {nombre}")



✅ Recortada: IMG_4827.JPG | Perfil izquierda | Expresión: Sin dentadura
✅ Recortada: IMG_4830.JPG | Girado izquierda | Expresión: Sin dentadura
✅ Recortada: IMG_4824.JPG | Frente | Expresión: Sin dentadura
✅ Recortada: IMG_4833.JPG | Girado derecha | Expresión: Sin dentadura
✅ Recortada: IMG_4835.JPG | Perfil derecha | Expresión: Sin dentadura
✅ Recortada: IMG_4828.JPG | Perfil izquierda | Expresión: Con dentadura
✅ Recortada: IMG_4831.JPG | Girado izquierda | Expresión: Con dentadura
✅ Recortada: IMG_4825.JPG | Frente | Expresión: Con dentadura
✅ Recortada: IMG_4834.JPG | Girado derecha | Expresión: Con dentadura
✅ Recortada: IMG_4836.JPG | Perfil derecha | Expresión: Con dentadura


In [51]:
from reportlab.lib.pagesizes import A4, landscape
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm
from PIL import Image
import pandas as pd
import os

# --- Rutas actualizadas ---
csv_path = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/notebooks/data.csv"
img_folder = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/fotos_finales"
output_pdf = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/output_fotos_finales.pdf"

# Leer CSV
df = pd.read_csv(csv_path)

# === PÁGINA 1 ===
c = canvas.Canvas(output_pdf, pagesize=A4)
w, h = A4

# Imagen frontal con dentadura
frontal = df[(df['posicion'] == 'Frente') & (df['expresion'] == 'Con dentadura')].iloc[0]
filename = f"cropped_Con_dentadura_{frontal['nombre_archivo']}"
img_path = os.path.join(img_folder, filename)

img = Image.open(img_path)
if img.width > img.height:
    img = img.rotate(90, expand=True)
    img.save(img_path)  # sobreescribe si fue rotada

# Dibujar imagen centrada
img_width = 12 * cm
aspect = img.height / img.width
img_height = img_width * aspect
c.drawImage(img_path, (w - img_width) / 2, h - img_height - 6*cm, width=img_width, height=img_height)

# Texto
c.setFont("Helvetica", 16)
c.drawCentredString(w / 2, h - img_height - 7.5*cm, "SEBASTIAN MORA AGUILERA")
c.setFont("Helvetica", 12)
c.drawString(2*cm, h - img_height - 8.7*cm, "EDAD: 10 AÑOS")
c.drawString(8*cm, h - img_height - 8.7*cm, "FECHA DE NACIMIENTO: 08/03/2015")
c.drawString(2*cm, h - img_height - 9.5*cm, "FECHA DE TOMA: 17/05/2025")
c.drawString(2*cm, h - img_height - 10.3*cm, "DRA. CLAUDIA RIVERO MARIN")
c.showPage()

# === PÁGINA 2 (horizontal) ===
c.setPageSize(landscape(A4))
w, h = landscape(A4)

orden = ["Perfil izquierda", "Girado izquierda", "Frente", "Girado derecha", "Perfil derecha"]
sin_dent = df[df['expresion'] == "Sin dentadura"].copy()
con_dent = df[df['expresion'] == "Con dentadura"].copy()

def get_ordered(filtrado, expresion):
    resultado = []
    for pos in orden:
        match = filtrado[filtrado['posicion'] == pos]
        if not match.empty:
            nombre = match.iloc[0]['nombre_archivo']
            archivo = f"cropped_{expresion.replace(' ', '_')}_{nombre}"
            resultado.append(archivo)
    return resultado

row1 = get_ordered(sin_dent, "Sin ßdentadura")
row2 = get_ordered(con_dent, "Con dentadura")

# Coordenadas
x_start = 1 * cm
img_size = 6 * cm
gap = (w - 2*cm - (5 * img_size)) / 3.5

# Fila 1: Sin dentadura
y1 = h - 4*cm - img_size
for i, filename in enumerate(row1):
    img_path = os.path.join(img_folder, filename)
    if os.path.exists(img_path):
        img = Image.open(img_path)
        if img.width > img.height:
            img = img.rotate(90, expand=True)
            img.save(img_path)
        x = x_start + i * (img_size + gap)
        c.drawImage(img_path, x, y1, width=img_size, height=img_size)

# Fila 2: Con dentadura
y2 = y1 - img_size - 1.8*cm
for i, filename in enumerate(row2):
    img_path = os.path.join(img_folder, filename)
    if os.path.exists(img_path):
        img = Image.open(img_path)
        if img.width > img.height:
            img = img.rotate(90, expand=True)
            img.save(img_path)
        x = x_start + i * (img_size + gap)
        c.drawImage(img_path, x, y2, width=img_size, height=img_size)

c.save()
print(f"✅ PDF generado con imágenes finales en: {output_pdf}")


✅ PDF generado con imágenes finales en: /Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/output_fotos_finales.pdf


# aux puntos nariz

In [32]:
import os
import ast
import pandas as pd
from PIL import Image, ImageDraw

# Rutas
csv_path = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/notebooks/data.csv"
img_folder = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/resized"
output_folder = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/resized_marcadas"

# Crear carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Leer CSV
df = pd.read_csv(csv_path)

# Iterar sobre las imágenes
for _, row in df.iterrows():
    nombre = row['nombre_archivo']
    nariz_raw = row['nariz']
    if pd.isna(nariz_raw):
        continue  # Saltar si no hay coordenada
    
    try:
        nariz = ast.literal_eval(nariz_raw)
        img_path = os.path.join(img_folder, nombre)
        if not os.path.exists(img_path):
            print(f"❌ Imagen no encontrada: {nombre}")
            continue
        
        img = Image.open(img_path)
        draw = ImageDraw.Draw(img)

        # Dibuja un círculo rojo de radio 8 px
        r = 8
        x, y = nariz
        draw.ellipse((x - r, y - r, x + r, y + r), fill='red')

        # Guardar imagen modificada
        out_path = os.path.join(output_folder, f"marcada_{nombre}")
        img.save(out_path)
        print(f"✅ Nariz marcada en: {nombre}")
    
    except Exception as e:
        print(f"⚠️ Error con {nombre}: {e}")


✅ Nariz marcada en: IMG_4836.JPG
✅ Nariz marcada en: IMG_4835.JPG
✅ Nariz marcada en: IMG_4834.JPG
✅ Nariz marcada en: IMG_4833.JPG
✅ Nariz marcada en: IMG_4831.JPG
✅ Nariz marcada en: IMG_4830.JPG
✅ Nariz marcada en: IMG_4829.JPG
✅ Nariz marcada en: IMG_4828.JPG
✅ Nariz marcada en: IMG_4827.JPG
✅ Nariz marcada en: IMG_4826.JPG
✅ Nariz marcada en: IMG_4825.JPG
✅ Nariz marcada en: IMG_4824.JPG
✅ Nariz marcada en: IMG_4823.JPG
